In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil.parser
import seaborn as sns
from IPython.display import Image
import os

fil_path = "../data/srt/srt_csv/"
dir_list = os.listdir(fil_path)

''' Settings '''
sns.set_style("white")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 3)

''' Load and concatinate CSV files into single dataframe'''

files=[]
for path in dir_list:
    files.append(
        pd.read_csv(fil_path+path,index_col=0)
    )
df_sub = pd.concat(files)

csv_names = ['../data/imdb_unique_prod-2018-04-21_0924.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path,index_col=0)
    )
df_movies = pd.concat(files)
df_movies = df_movies.loc[:, ~df_movies.columns.str.contains('^Unnamed')]

cols = df_movies.columns.tolist()
cols = cols[-2:] + cols[:-2]
df_movies = df_movies[cols]

# Drop null values
#df_movies = df_movies[(df_movies['imdb_popularity'].notnull())]
#df_movies['imdb_popularity'] = df_movies['imdb_popularity'].dropna()
#len(df_movies[df_movies['gross_usa'].isnull()])
#df_movies.head(5)

In [47]:
df_movies_missing = df_movies[df_movies['gross_usa'].isnull()]
df_movies_missing.head(5)

,title,year,budget,cast,color,country,description,director,genre,gross_usa,gross_usa_weekend,gross_worldwide,imdb_critic_reviews_total,imdb_popularity,imdb_rating,imdb_ratings_total,imdb_user_reviews_total,language,metacritic_rating,mpaa_rating,poster_image,production_co,release_date,runtime
1,A Woman Under the Influence,1974,NaN,"Peter Falk,Gena Rowlands,Fred Draper,Lady Rowl...",Color,USA,Peter Falk is a blue collar man trying to deal...,John Cassavetes,"Drama, Romance",NaN,NaN,NaN,85.0,NaN,8.2,"16,465",99,English,NaN,R,https://ia.media-imdb.com/images/M/MV5BYTRkZDR...,Faces,18 November 1974 (USA),155.0
4,Solaris,1972,"RUR 1,000,000","Natalya Bondarchuk,Donatas Banionis,Jüri Järve...",Black and White,Soviet Union,The Solaris mission has established a base on ...,Andrei Tarkovsky,"Drama, Mystery, Sci-Fi",NaN,NaN,NaN,173.0,"2,543",8.1,"63,833",253,Russian,90.0,PG,https://ia.media-imdb.com/images/M/MV5BZmY4Yjc...,Mosfilm,26 September 1972 (Czechoslovakia),167.0
7,Dillinger,1973,"$1,000,000","Warren Oates,Ben Johnson,Michelle Phillips,Clo...",Black and White,USA,After a shoot-out kills five FBI agents in Kan...,John Milius,"Action, Biography, Crime, Drama",NaN,NaN,NaN,35.0,NaN,7.0,"3,916",58,English,NaN,R,https://ia.media-imdb.com/images/M/MV5BMTcyMTE...,American International Pictures (AIP),8 November 1973 (Hong Kong),107.0
8,10 Rillington Place,1971,NaN,"Richard Attenborough,Judy Geeson,John Hurt,Pat...",Color,UK,"London, 1949.",Richard Fleischer,"Biography, Crime, Drama",NaN,NaN,NaN,55.0,NaN,7.6,"6,149",81,English,NaN,GP,https://ia.media-imdb.com/images/M/MV5BYzc4NDV...,Columbia Pictures,29 January 1971 (UK),111.0
9,The Homecoming,1973,NaN,"Paul Rogers,Ian Holm,Cyril Cusack,Terence Rigb...",Color,UK,Max is a surly pensioner who alternately vener...,Peter Hall,Drama,NaN,NaN,NaN,10.0,NaN,7.1,555,13,English,NaN,PG,https://ia.media-imdb.com/images/M/MV5BNzhkM2Y...,Cinévision Ltée,16 April 1977 (France),111.0


In [20]:
# Import numbers

csv_names = ['../data/numbers_worldwide-1-5.csv','../data/numbers_worldwide-6-50.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path,index_col=0)
    )
df_numbers = pd.concat(files)
df_numbers = df_numbers.reset_index()
df_numbers
#df_numbers = df_numbers.reset_index()
df_numbers['title'] = df_numbers['title'].str.replace(r"\(.*\)","")
df_numbers['title'] = df_numbers['title'].str.rstrip()
#df_numbers[df_numbers['title'].str.contains('Casino')]
df_numbers_titles = df_numbers['title']


In [21]:
# Attempt merge

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
df_movies = df_movies.sort_values(by='title')
df_numbers = df_numbers.sort_values(by='title')
#df_movies = df_movies[df_movies['title'] =="American Beauty"]
#df_numbers = df_numbers[df_numbers['title'] =="American Beauty"]
compare = pd.MultiIndex.from_product([df_movies['title'].head(200), df_numbers['title'].head(200)]).to_series()

def metrics(tup):
    return pd.Series([fuzz.ratio(*tup),
                      fuzz.token_sort_ratio(*tup)],
                     ['ratio', 'token'])

compared = compare.apply(metrics)
compared.sort_values(by='ratio',ascending=False)
compared



ratio  \
'G' Men                                                                 0   
                                                                        0   
                                                                        0   
                                                                        0   
                *batteries not included                                13   
                ...And Justice For All                                 14   
                10                                                      0   
                10 Cloverfield Lane                                    23   
                10 Things I Hate About You                             12   
                10,000 B.C.                                            11   
...                                                                   ...   
Bicycle Thieves Airport '77                                            15   
                Airport 1975                                           15   
                Akeelah and the Bee                                    29   
                Aku no Kyoten                                          14   
                Aladdin                                                 9   
                Alatriste                                              17   
                Alex Cross                                             32   
                Alex Rider: Operation Stormbreaker                     24   
                Alexander                                              25   
                Alexander and the Terrible, Horrible, No Good, ...     19   

                                                                    token  
'G' Men                                                                 0  
                                                                        0  
                                                                        0  
                                                                        0  
                *batteries not included                                22  
                ...And Justice For All                                 17  
                10                                                      0  
                10 Cloverfield Lane                                    25  
                10 Things I Hate About You                             13  
                10,000 B.C.                                            13  
...                                                                   ...  
Bicycle Thieves Airport '77                                            16  
                Airport 1975                                           15  
                Akeelah and the Bee                                    35  
                Aku no Kyoten                                          21  
                Aladdin                                                 9  
                Alatriste                                              17  
                Alex Cross                                             32  
                Alex Rider: Operation Stormbreaker                     29  
                Alexander                                              25  
                Alexander and the Terrible, Horrible, No Good, ...     28  

[40000 rows x 2 columns]

In [31]:
def fuzzy_match(x, choices, scorer, cutoff):
    return process.extractOne(
        x, choices=choices, scorer=scorer, score_cutoff=cutoff
    )

matching_results = df_movies['title'].head(1000).apply(
    fuzzy_match,
    args=(
        df_numbers['title'].head(1000), 
        fuzz.ratio,
        2
    )
)
matching_results

0    Dancer in the Dark
0                    55
0                  3381
Name: title, dtype: object

In [40]:
matching_results

exact_matches = df_movies['title'].isin(
    df_numbers['title'].tolist()
)
exact_matches = df_movies[exact_matches]
exact_matches

results = exact_matches
results['title'] = results['title']
results['match_type'] = 'exact'
results

/Users/justinblinder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/justinblinder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,title,year,budget,cast,color,country,description,director,genre,gross_usa,gross_usa_weekend,gross_worldwide,imdb_critic_reviews_total,imdb_popularity,imdb_rating,imdb_ratings_total,imdb_user_reviews_total,language,metacritic_rating,mpaa_rating,poster_image,production_co,release_date,runtime,match_type
608,101 Dalmatians,1996,"$75,000,000","Glenn Close,Jeff Daniels,Joely Richardson,Joan...",Color,USA,After a wild chase in the bustling streets of ...,Stephen Herek,"Adventure, Comedy, Crime, Family","$136,189,294","$45,073,479","$320,689,294",56.0,"2,390",5.7,"85,512",78,English,NaN,G,https://ia.media-imdb.com/images/M/MV5BZDY3NDQ...,Walt Disney Pictures,27 November 1996 (USA),103.0,exact
3,12 Strong,2018,NaN,"Chris Hemsworth,Michael Shannon,Michael Peña,N...",Color,USA,"The True Story of the Army's Special Forces ""G...",Nicolai Fuglsig,"Action, Drama, History, War","$45,495,662,","$15,815,025","$35,159,610,",122.0,15,6.7,"16,345",123,English,NaN,R,https://ia.media-imdb.com/images/M/MV5BNTEzMjk...,Alcon Entertainment,19 January 2018 (USA),130.0,exact
197,12 Years a Slave,2013,"$20,000,000","Chiwetel Ejiofor,Dwight Henry,Dickie Gravois,B...",Color,USA,Based on an incredible true story of one man's...,Steve McQueen,"Biography, Drama, History","$56,671,993","$923,715","$187,733,202",601.0,667,8.1,"533,269",743,English,NaN,R,https://ia.media-imdb.com/images/M/MV5BZjFkOGN...,Regency Enterprises,8 November 2013 (USA),134.0,exact
873,2 Guns,2013,"$61,000,000","Denzel Washington,Mark Wahlberg,Paula Patton,E...",Color,USA,"Robert Trench, an undercover DEA agent, takes ...",Baltasar Kormákur,"Action, Comedy, Crime, Thriller","$75,612,460","$27,059,130","$131,940,411",270.0,"2,067",6.7,"173,119",178,English,NaN,R,https://ia.media-imdb.com/images/M/MV5BNTQ5MTg...,Universal Pictures,2 August 2013 (USA),109.0,exact
752,"20,000 Leagues Under the Sea",1916,"$200,000","Dan Hanlon,Edna Pendleton,Curtis Benton,Allen ...",Black and White,USA,Captain Nemo has built a fantastic submarine f...,Stuart Paton,"Action, Adventure, Sci-Fi","$8,000,000",NaN,NaN,16.0,NaN,7.0,"1,293",23,English,NaN,Passed,https://ia.media-imdb.com/images/M/MV5BMTQ0OTk...,Williamson Submarine Film Corporation,24 December 1916 (USA),105.0,exact
269,2001: A Space Odyssey,1968,"$12,000,000","Keir Dullea,Gary Lockwood,William Sylvester,Da...",Color,UK,"""2001"" is a story of evolution. Sometime in th...",Stanley Kubrick,"Adventure, Sci-Fi","$56,954,992",NaN,"$190,700,000",262.0,288,8.3,"498,654","1,855",English,82.0,G,https://ia.media-imdb.com/images/M/MV5BMmNlYzR...,Metro-Goldwyn-Mayer (MGM),12 May 1968 (UK),149.0,exact
725,2012,2009,"$200,000,000","John Cusack,Amanda Peet,Chiwetel Ejiofor,Thand...",Color,USA,"Dr. Adrian Helmsley, part of a worldwide geoph...",Roland Emmerich,"Action, Adventure, Sci-Fi","$166,112,167,","$65,237,614","$769,679,473,",370.0,887,5.8,"312,067","1,084",English,NaN,PG-13,https://ia.media-imdb.com/images/M/MV5BMTY0MjE...,Columbia Pictures,13 November 2009 (USA),158.0,exact
850,21,2008,"$35,000,000","Jim Sturgess,Kevin Spacey,Kate Bosworth,Aaron ...",Color,USA,"Ben Campbell is a young, highly intelligent, s...",Robert Luketic,"Crime, Drama, Thriller","$81,159,365","$24,105,943","$157,927,340",146.0,"2,219",6.8,"205,834",286,English,NaN,PG-13,https://ia.media-imdb.com/images/M/MV5BMjAyNTU...,Columbia Pictures,28 March 2008 (USA),123.0,exact
617,21 Jump Street,2012,"$42,000,000","Jonah Hill,Channing Tatum,Brie Larson,Dave Fra...",Color,USA,"In high school, Schmidt (",Phil Lord,"Action, Comedy, Crime","$138,447,667,","$36,302,612","$201,585,328,",376.0,"1,010",7.2,"454,033",364,English,69.0,R,https://ia.media-imdb.com/images/M/MV5BMTc3NzQ...,Columbia Pictures,16 March 2012 (USA),109.0,exact
902,27 Dresses,2008,"$30,000,000","Brian Kerwin,Charli Barcena,Peyton List,Jane P...",Color,USA,Two things about Jane: she never says no to he...,Anne Fletcher,"Comedy, Romance","$76,808,654,","$27,442,040","$160,259,319",174.0,631,6.1,"137,197",154,English,NaN,PG-13,http

In [45]:
# pip install fuzzywuzzy
from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

# List for dicts for easy dataframe creation
dict_list = []
# iterating over our players without salaries found above
for name in df_numbers['title']:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, df_movies['title'], 75)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"title" : name})
    dict_.update({"match_name" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)


merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

,match_name,score,title
0,,-1,
1,,-1,
2,,-1,
3,,-1,
4,,-1,*batteries not included
5,,-1,...And Justice For All
6,,-1,10
7,,-1,10 Cloverfield Lane
8,,-1,10 Things I Hate About You
9,"10,000 BC",90,"10,000 B.C."


In [42]:
len(matching_results)

1000

In [22]:
compare.apply(metrics).unstack().idxmax().unstack(0)

ValueError: Index contains duplicate entries, cannot reshape

In [23]:
compare.apply(metrics).unstack(0).idxmax().unstack(0)

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
from currency_converter import CurrencyConverter
from re import sub
import math

c = CurrencyConverter()

def parse_currency(value):
    value = str(value)
    currency_type = sub(r'[?!^\d.]', '', value ).replace(',','').rstrip()
    currency_amount = sub(r'[^\d.]', '', value)
    if currency_type == "$":
        currency_type = 'USD'
    if currency_type == "€":
        currency_type = 'EUR'
    if currency_type == "£":
        currency_type = 'GBP'
    if currency_type == "RUR":
        currency_type = 'USD'
    if currency_type == "FRF":
        currency_type = 'EUR'
    if currency_type == "DEM":
        currency_type = 'EUR'
    if currency_type == "":
        currency_type = 'USD'        
    return int(math.ceil(c.convert(currency_amount, currency_type, 'USD')))
    
        


#df_movies = df_movies[df_movies['gross_usa'].notnull()]
#df_movies['gross_usa'] = df_movies['gross_usa'].apply(clean_imdb_box_office)
df_movies['budget'] = df_movies['budget'].apply(parse_currency)
df_movies['gross_usa'] = df_movies['gross_usa'].apply(parse_currency)
df_movies['gross_usa_weekend'] = df_movies['gross_usa_weekend'].apply(parse_currency)
df_movies['gross_worldwide'] = df_movies['gross_worldwide'].apply(parse_currency)

df_movies.head(5)
#df_movies

In [ ]:
# Calculate length of line dialogue 
df_sub['line_duration'] = df_sub['endTime'] - df_sub['startTime']

# Calculate length of line dialogue
# THIS LINE TOTALLY CHNAGES GRAPHS, IF SHIFT IS 1 vs -1
df_sub['line_delta'] = df_sub['startTime'] - df_sub['endTime'].shift(1)
df_sub['line_delta'] = df_sub['line_delta'].fillna(0).astype(int)
df_movies['genre'] = df_movies['genre'].dropna()

# Fix dupliacte index TODO: WHY IS THIS HAPPENING?
#df_movies = df_movies.reset_index()

# Create dummy variables
#df_movies['genre'] = df_movies['genre'].str.get_dummies(sep=',').add_prefix('genre_')
#df_movies[] = pd.get_dummies(df_movies['genre'])

# Merge subtitle and movie dataframe
df_test = pd.merge(df_sub, df_movies[['title','year','genre','budget','imdb_rating','imdb_popularity','color','director','country','metacritic_rating','mpaa_rating','gross_usa','gross_usa_weekend','gross_worldwide']], on='title')
df_test = df_movies.groupby('title').agg(lambda x: x[x>0].mean()).reset_index()
#df_movies.index.get_duplicates()
df_test
#df_movies

In [ ]:
from matplotlib import rcParams

plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
rcParams['figure.figsize'] = 11.7,8.27
#df_test = df_test[(df_test['budget'] < 500000000) & (df_test['gross_usa'] < 4000000000) & (df_test['gross_usa_weekend'] < 500000000) & (df_test['gross_worldwide'] < 100000000) & (df_test['imdb_rating'] < 8)]

sns.lmplot(x='budget',y='line_duration',data=df_test,fit_reg=True) 
sns.lmplot(x='budget',y='line_delta',data=df_test,fit_reg=True) 


sns.lmplot(x='gross_usa',y='line_duration',data=df_test,fit_reg=True) 
sns.lmplot(x='gross_usa',y='line_delta',data=df_test,fit_reg=True) 


sns.lmplot(x='gross_usa_weekend',y='line_duration',data=df_test,fit_reg=True) 
sns.lmplot(x='gross_usa_weekend',y='line_delta',data=df_test,fit_reg=True) 

sns.lmplot(x='gross_worldwide',y='line_duration',data=df_test,fit_reg=True) 
sns.lmplot(x='gross_worldwide',y='line_delta',data=df_test,fit_reg=True) 

sns.lmplot(x='imdb_rating',y='line_duration',data=df_test,fit_reg=True) 
sns.lmplot(x='imdb_rating',y='line_delta',data=df_test,fit_reg=True) 

sns.lmplot(x='imdb_popularity',y='line_duration',data=df_test,fit_reg=True) 
sns.lmplot(x='imdb_popularity',y='line_delta',data=df_test,fit_reg=True) 


In [ ]:
df_r = df_test[['year','startTime','endTime','line_duration','line_delta','imdb_rating']]
df_r

In [ ]:
''' Merge tables for regression'''
df_reg = pd.merge(df_sub, df_mov[['title','year','genre','budget','imdb_rating','imdb_popularity','color','director','country','metacritic_rating','mpaa_rating']], on='title')
df_reg.head(5)

In [ ]:

df_test.corr()

In [ ]:

import seaborn as sns
sns.set(font_scale=2)
sns.pairplot(df_test[df_test['year'] > 1950],size = 4)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std
df_r = df_r[['year','line_duration','line_delta']]
# Create an empty model
lr = LinearRegression()
df_r = df_r[df_r['year'] > 1940]
# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X2 + X3 + X4 + X5 + X6 model
X = df_r.iloc[:,1:]
# Choose the response variable(s)
y = df_r.iloc[:,0]
# Fit the model to the full dataset
lr.fit(X,y)
# Print out the R^2 for the model against the full dataset


model = ols("line_duration ~ year", data=df_r).fit()
fig = plt.figure(figsize=(15,8))

# pass in the model as the first parameter, then specify the 
# predictor variable we want to analyze
fig = sm.graphics.plot_regress_exog(model, "year", fig=fig)
lr.score(X,y)

In [ ]:
lsm = smf.ols('year~ line_delta + line_duration + imdb_rating', data = df_r)
fit1 = lsm.fit()
fit1.summary()


In [ ]:
fit1.resid.plot(style='o', figsize=(12,8))